In [3]:
#After making changes in seed customer selection, the code is working fine.
#This have logic of paired orders and paired pickup orders.



from datetime import timedelta
import json
import time
import csv
import os

def process_vrp_file(file_path, result_folder):
    # Load the JSON input from a file
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Extract data from JSON
    distance_matrix = data['distance_matrix']
    time_matrix = data['time_matrices'][0]
    demands = data['demands']
    serve_time = data['serve_time']
    vehicle_capacities = data['vehicle_capacities']
    vehicle_max_weight = data['vehicle_capacities']
    fixed_cost = data['fixed_cost']
    cost_per_km = data['cost_per_km']
    contract_distance = data['contract_distance']
    vehicle_volume = data['vehicle_volume']
    cost_per_minute = data['cost_per_minute']
    variable_cost_per_minute = data['variable_cost_per_minute']
    contract_time = data['contract_time']
    vehicle_average_speed = data['vehicle_average_speed']
    order_volume = data['order_volume']
    vehicle_volumes = data['vehicle_volumes']
    vehicle_contract_code = data['vehicle_contract_code']
    vehicle_max_travel_distance = data['vehicle_max_travel_distance']
    vehicle_max_travel_time = data['vehicle_max_available_time']
    order_types = data['order_type']
    depot = data['depot']
    order_id=data['order_id']
    vehicle_max_order=data["vehicle_max_order"]
    order_priority=data["order_priority"]
   
    num_vehicles = data['num_vehicles']
    paired_orders = data['paired_order']
    paired_dict = {pair[0]: pair[1] for pair in paired_orders}

    
    paired_pickup_orders=data['paired_pickup_orders']
    pickup_order={pair[0]:pair[1] for pair in paired_pickup_orders}

    paired_dropoff_orders=data['paired_dropoff_orders']
    pickup_dropoff_orders={pair[0]:pair[1] for pair in paired_dropoff_orders}

    

    start_time = time.time()

    def parse_time_windows(time_window_list):
        return [datetime.fromisoformat(tw) for tw in time_window_list]

    order_time_window_start = parse_time_windows(data["order_time_window_start"])
    order_time_window_end = parse_time_windows(data["order_time_window_end"])
    vehicle_time_window_start = parse_time_windows(data["vehicle_time_window_start"])
    vehicle_time_window_end = parse_time_windows(data["vehicle_time_window_end"])

    order_time_windows = list(zip(order_time_window_start, order_time_window_end))

    # Combine vehicle time windows into list of tuples
    vehicle_time_windows = list(zip(vehicle_time_window_start, vehicle_time_window_end))

   


    def select_seed_customers_by_proximity(distance_matrix, num_vehicles):
        depot_index = 0
        distances = [(i, distance_matrix[depot_index][i]) for i in range(1, len(distance_matrix))]
        distances.sort(key=lambda x: x[1])
        return [distances[i][0] for i in range(min(num_vehicles, len(distances)))]


    def calculate_route_cost(route, vehicle_index, distance_matrix, time_matrix):
        total_distance = 0
        total_time = 0

        if len(route) > 0:  
            depot_index = 0
            total_distance += distance_matrix[depot_index][route[0]]  
            total_time += time_matrix[depot_index][route[0]]

        for i in range(len(route) - 1):
            total_distance += distance_matrix[route[i]][route[i + 1]]
            total_time += time_matrix[route[i]][route[i + 1]]
            total_time += serve_time[route[i]]

        if len(route) > 0:  
            total_distance += distance_matrix[route[-1]][depot_index]  
            total_time += time_matrix[route[-1]][depot_index]
            total_time += serve_time[route[-1]]

        excess_distance = max(0, total_distance - contract_distance[vehicle_index])
        variable_distance_cost = total_distance * cost_per_km[vehicle_index]

        if excess_distance > 0:
            variable_distance_cost += excess_distance * cost_per_km[vehicle_index]

        total_cost = (fixed_cost[vehicle_index] +
                      variable_distance_cost +
                      (total_time * cost_per_minute[vehicle_index]))

        return total_cost, total_distance, total_time

    def local_cheapest_insertion_with_paired_orders(
        distance_matrix, time_matrix, num_vehicles, demands, vehicle_capacities,
        order_volume, vehicle_max_travel_time, serve_time, vehicle_max_weight,
        vehicle_volume, order_types, paired_orders, paired_pickup_orders,
        paired_dropoff_orders, vehicle_time_windows, order_time_windows,
        vehicle_max_travel_distance, vehicle_max_order):
    
        routes = [[] for _ in range(num_vehicles)]
        skipped_orders = []
        unvisited_customers = set(range(1, len(demands)))  # Assuming customer IDs start from 1

        # Dictionaries for paired orders
        paired_pickup_orders = {pair[0]: pair[1] for pair in paired_pickup_orders}
        pickup_dropoff_orders = {pair[0]: pair[1] for pair in paired_dropoff_orders}
        pairing_pickup_to_dropoff = {pair[0]: pair[1] for pair in paired_orders}
        paired_dropoff_dict = {pair[0]: pair[1] for pair in paired_orders}

        # Function to assign orders to a vehicle
        def assign_orders_to_vehicle(v, current_node, unvisited_customers, skipped_orders, routes):
            vehicle_capacity = vehicle_capacities[v]
            vehicle_start_time, vehicle_end_time = vehicle_time_windows[v]
            current_time = vehicle_start_time
            current_route = []
            pickup_weight = 0
            dropoff_weight = 0
            available_weight = vehicle_capacity

            while unvisited_customers:
                # Find the nearest unvisited customer that can be assigned
                nearest_customer = None
                min_distance = float('inf')

                for customer in unvisited_customers:
                    if distance_matrix[current_node][customer] < min_distance:
                        nearest_customer = customer
                        min_distance = distance_matrix[current_node][customer]

                if nearest_customer is None:
                    break  # No more customers can be assigned

                order = nearest_customer
                ot = order_types[order - 1]  # Adjust for zero-based indexing
                weight = demands[order]

                if ot == "pickup":
                    # Handle paired pickup orders
                    if order in paired_pickup_orders:
                        paired_pickup = paired_pickup_orders[order]
                        paired_weight = demands[paired_pickup]

                        if paired_pickup not in unvisited_customers:
                            # Paired pickup already assigned
                            skipped_orders.extend([order, paired_pickup])
                            print(f"Skipped Pickup Order {order} and Paired Pickup {paired_pickup} as paired pickup is already assigned.")
                            unvisited_customers.remove(order)
                            continue

                        total_pickup_weight = weight + paired_weight
                        if pickup_weight + total_pickup_weight <= vehicle_capacity and available_weight >= total_pickup_weight:
                            # Assign both pickup and its paired pickup
                            current_route.extend([order, paired_pickup])
                            pickup_weight += total_pickup_weight
                            available_weight -= total_pickup_weight
                            unvisited_customers.remove(order)
                            unvisited_customers.remove(paired_pickup)
                            current_node = paired_pickup  # Move to the paired pickup
                            print(f"Assigned Paired Pickup Orders {order} and {paired_pickup} to current route.")
                        else:
                            # Cannot assign due to capacity constraints
                            skipped_orders.extend([order, paired_pickup])
                            print(f"Skipped Paired Pickup Orders {order} and {paired_pickup} due to capacity constraints.")
                            unvisited_customers.remove(order)
                            continue

                    # Handle pickup paired with dropoff
                    elif order in pairing_pickup_to_dropoff:
                        paired_dropoff = pairing_pickup_to_dropoff[order]
                        paired_dropoff_weight = demands[paired_dropoff]

                        if paired_dropoff not in unvisited_customers:
                            # Paired dropoff already assigned
                            skipped_orders.extend([order, paired_dropoff])
                            print(f"Skipped Pickup Order {order} and Paired Dropoff {paired_dropoff} as paired dropoff is already assigned.")
                            unvisited_customers.remove(order)
                            continue

                        if pickup_weight + weight + paired_dropoff_weight <= vehicle_capacity and available_weight >= (weight + paired_dropoff_weight):
                            # Assign pickup and its paired dropoff
                            current_route.extend([order, paired_dropoff])
                            unvisited_customers.remove(order)
                            available_weight -= weight
                            pickup_weight += weight
                            unvisited_customers.remove(paired_dropoff)
                            current_node = paired_dropoff  # Move to the paired dropoff
                            print(f"Assigned Pickup Order {order} and its Paired Dropoff Order {paired_dropoff} to current route.")
                        else:
                            # Cannot assign due to capacity constraints
                            skipped_orders.extend([order, paired_dropoff])
                            print(f"Skipped Pickup Order {order} and Dropoff Order {paired_dropoff} due to capacity constraints.")
                            unvisited_customers.remove(order)
                            continue

                    else:
                        # Unpaired pickup
                        if pickup_weight + weight <= vehicle_capacity and available_weight >= weight:
                            # Assign unpaired pickup
                            current_route.append(order)
                            pickup_weight += weight
                            available_weight -= weight
                            unvisited_customers.remove(order)
                            current_node = order  # Move to the assigned customer
                            print(f"Assigned Unpaired Pickup Order {order} to current route.")
                        else:
                            # Cannot assign due to capacity constraints
                            skipped_orders.append(order)
                            print(f"Skipped Unpaired Pickup Order {order} due to capacity constraints.")
                            unvisited_customers.remove(order)
                            continue

                elif ot == "dropoff":
                    # Handle paired dropoff orders
                    paired_dropoff = None
                    for key, value in paired_dropoff_dict.items():
                        if value == order:
                            paired_dropoff = key
                            break
                        elif key == order:
                            paired_dropoff = value
                            break

                    if paired_dropoff:
                        paired_dropoff_weight = demands[paired_dropoff]

                        if paired_dropoff not in unvisited_customers:
                            # Paired dropoff already assigned
                            skipped_orders.extend([order, paired_dropoff])
                            print(f"Skipped Dropoff Order {order} and Paired Dropoff {paired_dropoff} as paired dropoff is already assigned.")
                            unvisited_customers.remove(order)
                            continue

                        if dropoff_weight + weight + paired_dropoff_weight <= vehicle_capacity and available_weight >= (weight + paired_dropoff_weight):
                            # Assign both dropoff and its paired dropoff
                            current_route.extend([order, paired_dropoff])
                            dropoff_weight += (weight + paired_dropoff_weight)
                            unvisited_customers.remove(order)
                            unvisited_customers.remove(paired_dropoff)
                            current_node = paired_dropoff  # Move to the paired dropoff
                            print(f"Assigned Paired Dropoff Orders {order} and {paired_dropoff} to current route.")
                        else:
                            # Cannot assign due to capacity constraints
                            skipped_orders.extend([order, paired_dropoff])
                            print(f"Skipped Paired Dropoff Orders {order} and {paired_dropoff} due to capacity constraints.")
                            unvisited_customers.remove(order)
                            continue
                    else:
                        # Unpaired dropoff
                        if dropoff_weight + weight <= vehicle_capacity and available_weight >= weight:
                            # Assign unpaired dropoff
                            current_route.append(order)
                            dropoff_weight += weight
                            unvisited_customers.remove(order)
                            current_node = order  # Move to the assigned customer
                            print(f"Assigned Unpaired Dropoff Order {order} to current route.")
                        else:
                            # Cannot assign due to capacity constraints
                            skipped_orders.append(order)
                            print(f"Skipped Unpaired Dropoff Order {order} due to capacity constraints.")
                            unvisited_customers.remove(order)
                            continue

            if current_route:
                routes[v] = current_route
                print(f"Finalized Route for Vehicle {v}: {current_route}")

        # Iterate over vehicles one by one
        for v in range(num_vehicles):
            if not unvisited_customers:
                print(f"All customers have been assigned. No unvisited customers left for vehicle {v}.")
                break  # All customers have been assigned
            assign_orders_to_vehicle(v, 0, unvisited_customers, skipped_orders, routes)

        # Reassign skipped orders to the remaining vehicles in a round-robin manner
        unused_vehicles = [v for v in range(num_vehicles) if not routes[v]]
        for v in unused_vehicles:
            if not skipped_orders:
                break  # No skipped orders left
            assign_orders_to_vehicle(v, 0, set(skipped_orders), skipped_orders, routes)

        return routes, skipped_orders

    



    final_routes = local_cheapest_insertion_with_paired_orders(distance_matrix, time_matrix, num_vehicles, demands, vehicle_capacities, order_volume, vehicle_max_travel_time, serve_time, vehicle_max_weight, vehicle_volume, order_types, paired_orders,paired_pickup_orders,paired_dropoff_orders,vehicle_time_windows, order_time_windows,vehicle_max_travel_distance,vehicle_max_order)
    print(final_routes)





                  

    # def local_cheapest_insertion_with_paired_orders(
    #         distance_matrix, time_matrix, num_vehicles, demands, vehicle_capacities,
    #         order_volume, vehicle_max_travel_time, serve_time, vehicle_max_weight,
    #         vehicle_volume, order_types, paired_orders, paired_pickup_orders,
    #         paired_dropoff_orders, vehicle_time_windows, order_time_windows,
    #         vehicle_max_travel_distance, vehicle_max_order):
        
    #     routes = [[] for _ in range(num_vehicles)]
    #     skipped_orders = []
    #     unvisited_customers = set(range(1, len(demands)))  # Assuming customer IDs start from 1

    #     # Dictionaries for paired orders
    #     paired_pickup_orders = {pair[0]: pair[1] for pair in paired_pickup_orders}
    #     pickup_dropoff_orders = {pair[0]: pair[1] for pair in paired_dropoff_orders}
    #     pairing_pickup_to_dropoff = {pair[0]: pair[1] for pair in paired_orders}
    #     paired_dropoff_dict = {pair[0]: pair[1] for pair in paired_orders}

    #     # Function to assign orders to a vehicle
    #     def assign_orders_to_vehicle(v, current_node, unvisited_customers, skipped_orders, routes):
    #         vehicle_capacity = vehicle_capacities[v]
    #         vehicle_start_time, vehicle_end_time = vehicle_time_windows[v]
    #         current_time = vehicle_start_time
    #         current_route = []
    #         pickup_weight = 0
    #         dropoff_weight = 0
    #         available_weight = vehicle_capacity

    #         while unvisited_customers:
    #             # Find the nearest unvisited customer that can be assigned
    #             nearest_customer = None
    #             min_distance = float('inf')

    #             for customer in unvisited_customers:
    #                 if distance_matrix[current_node][customer] < min_distance:
    #                     nearest_customer = customer
    #                     min_distance = distance_matrix[current_node][customer]

    #             if nearest_customer is None:
    #                 break  # No more customers can be assigned

    #             order = nearest_customer
    #             ot = order_types[order - 1]  # Adjust for zero-based indexing
    #             weight = demands[order]

    #             if ot == "pickup":
    #                 # Handle paired pickup orders
    #                 if order in paired_pickup_orders:
    #                     paired_pickup = paired_pickup_orders[order]
    #                     paired_weight = demands[paired_pickup]

    #                     if paired_pickup not in unvisited_customers:
    #                         # Paired pickup already assigned
    #                         skipped_orders.extend([order, paired_pickup])
    #                         print(f"Skipped Pickup Order {order} and Paired Pickup {paired_pickup} as paired pickup is already assigned.")
    #                         unvisited_customers.remove(order)
    #                         continue

    #                     total_pickup_weight = weight + paired_weight
    #                     if pickup_weight + total_pickup_weight <= vehicle_capacity and available_weight >= total_pickup_weight:
    #                         # Assign both pickup and its paired pickup
    #                         current_route.extend([order, paired_pickup])
    #                         pickup_weight += total_pickup_weight
    #                         available_weight -= total_pickup_weight
    #                         unvisited_customers.remove(order)
    #                         unvisited_customers.remove(paired_pickup)
    #                         current_node = paired_pickup  # Move to the paired pickup
    #                         print(f"Assigned Paired Pickup Orders {order} and {paired_pickup} to current route.")
    #                     else:
    #                         # Cannot assign due to capacity constraints
    #                         skipped_orders.extend([order, paired_pickup])
    #                         print(f"Skipped Paired Pickup Orders {order} and {paired_pickup} due to capacity constraints.")
    #                         unvisited_customers.remove(order)
    #                         continue

    #                 # Handle pickup paired with dropoff
    #                 elif order in pairing_pickup_to_dropoff:
    #                     paired_dropoff = pairing_pickup_to_dropoff[order]
    #                     paired_dropoff_weight = demands[paired_dropoff]

    #                     if paired_dropoff not in unvisited_customers:
    #                         # Paired dropoff already assigned
    #                         skipped_orders.extend([order, paired_dropoff])
    #                         print(f"Skipped Pickup Order {order} and Paired Dropoff {paired_dropoff} as paired dropoff is already assigned.")
    #                         unvisited_customers.remove(order)
    #                         continue

    #                     if pickup_weight + weight + paired_dropoff_weight <= vehicle_capacity and available_weight >= (weight + paired_dropoff_weight):
    #                         # Assign pickup and its paired dropoff
    #                         current_route.extend([order, paired_dropoff])
    #                         unvisited_customers.remove(order)
    #                         available_weight -= weight
    #                         pickup_weight += weight
    #                         unvisited_customers.remove(paired_dropoff)
    #                         current_node = paired_dropoff  # Move to the paired dropoff
    #                         print(f"Assigned Pickup Order {order} and its Paired Dropoff Order {paired_dropoff} to current route.")
    #                     else:
    #                         # Cannot assign due to capacity constraints
    #                         skipped_orders.extend([order, paired_dropoff])
    #                         print(f"Skipped Pickup Order {order} and Dropoff Order {paired_dropoff} due to capacity constraints.")
    #                         unvisited_customers.remove(order)
    #                         continue

    #                 else:
    #                     # Unpaired pickup
    #                     if pickup_weight + weight <= vehicle_capacity and available_weight >= weight:
    #                         # Assign unpaired pickup
    #                         current_route.append(order)
    #                         pickup_weight += weight
    #                         available_weight -= weight
    #                         unvisited_customers.remove(order)
    #                         current_node = order  # Move to the assigned customer
    #                         print(f"Assigned Unpaired Pickup Order {order} to current route.")
    #                     else:
    #                         # Cannot assign due to capacity constraints
    #                         skipped_orders.append(order)
    #                         print(f"Skipped Unpaired Pickup Order {order} due to capacity constraints.")
    #                         unvisited_customers.remove(order)
    #                         continue

    #             elif ot == "dropoff":
    #                 # Handle paired dropoff orders
    #                 paired_dropoff = None
    #                 for key, value in paired_dropoff_dict.items():
    #                     if value == order:
    #                         paired_dropoff = key
    #                         break
    #                     elif key == order:
    #                         paired_dropoff = value
    #                         break

    #                 if paired_dropoff:
    #                     paired_dropoff_weight = demands[paired_dropoff]

    #                     if paired_dropoff not in unvisited_customers:
    #                         # Paired dropoff already assigned
    #                         skipped_orders.extend([order, paired_dropoff])
    #                         print(f"Skipped Dropoff Order {order} and Paired Dropoff {paired_dropoff} as paired dropoff is already assigned.")
    #                         unvisited_customers.remove(order)
    #                         continue

    #                     if dropoff_weight + weight + paired_dropoff_weight <= vehicle_capacity and available_weight >= (weight + paired_dropoff_weight):
    #                         # Assign both dropoff and its paired dropoff
    #                         current_route.extend([order, paired_dropoff])
    #                         dropoff_weight += (weight + paired_dropoff_weight)
    #                         unvisited_customers.remove(order)
    #                         unvisited_customers.remove(paired_dropoff)
    #                         current_node = paired_dropoff  # Move to the paired dropoff
    #                         print(f"Assigned Paired Dropoff Orders {order} and {paired_dropoff} to current route.")
    #                     else:
    #                         # Cannot assign due to capacity constraints
    #                         skipped_orders.extend([order, paired_dropoff])
    #                         print(f"Skipped Paired Dropoff Orders {order} and {paired_dropoff} due to capacity constraints.")
    #                         unvisited_customers.remove(order)
    #                         continue
    #                 else:
    #                     # Unpaired dropoff
    #                     if dropoff_weight + weight <= vehicle_capacity and available_weight >= weight:
    #                         # Assign unpaired dropoff
    #                         current_route.append(order)
    #                         dropoff_weight += weight
    #                         unvisited_customers.remove(order)
    #                         current_node = order  # Move to the assigned customer
    #                         print(f"Assigned Unpaired Dropoff Order {order} to current route.")
    #                     else:
    #                         # Cannot assign due to capacity constraints
    #                         skipped_orders.append(order)
    #                         print(f"Skipped Unpaired Dropoff Order {order} due to capacity constraints.")
    #                         unvisited_customers.remove(order)
    #                         continue

    #         if current_route:
    #             routes[v] = current_route
    #             print(f"Finalized Route for Vehicle {v}: {current_route}")

    #     # Iterate over vehicles one by one
    #     for v in range(num_vehicles):
    #         if not unvisited_customers:
    #             print(f"All customers have been assigned. No unvisited customers left for vehicle {v}.")
    #             break  # All customers have been assigned
    #         assign_orders_to_vehicle(v, 0, unvisited_customers, skipped_orders, routes)

    #     # Reassign skipped orders to the remaining vehicles in a round-robin manner
    #     unused_vehicles = [v for v in range(num_vehicles) if not routes[v]]
    #     for v in unused_vehicles:
    #         if not skipped_orders:
    #             break  # No skipped orders left
    #         assign_orders_to_vehicle(v, 0, set(skipped_orders), skipped_orders, routes)

    #     return routes, skipped_orders





 
    # def local_cheapest_insertion_with_paired_orders(
    #         distance_matrix, time_matrix, num_vehicles, demands, vehicle_capacities,
    #         order_volume, vehicle_max_travel_time, serve_time, vehicle_max_weight,
    #         vehicle_volume, order_types, paired_orders, paired_pickup_orders,
    #         paired_dropoff_orders, vehicle_time_windows, order_time_windows,
    #         vehicle_max_travel_distance, vehicle_max_order):
        
    #     routes = [[] for _ in range(num_vehicles)]
    #     skipped_orders = []
    #     unvisited_customers = set(range(1, len(demands)))  # Assuming customer IDs start from 1

    #     # Dictionaries for paired orders
    #     paired_pickup_orders = {pair[0]: pair[1] for pair in paired_pickup_orders}
    #     pickup_dropoff_orders = {pair[0]: pair[1] for pair in paired_dropoff_orders}
    #     pairing_pickup_to_dropoff = {pair[0]: pair[1] for pair in paired_orders}
    #     paired_dropoff_dict = {pair[0]: pair[1] for pair in paired_orders}

    #     # Iterate over vehicles one by one
    #     for v in range(num_vehicles):
    #         if not unvisited_customers:
    #             print(f"All customers have been assigned. No unvisited customers left for vehicle {v}.")
    #             break  # All customers have been assigned

    #         # Get vehicle's capacity, travel constraints, and time window
    #         vehicle_capacity = vehicle_capacities[v]
    #         vehicle_max_time = vehicle_max_travel_time[v]
    #         vehicle_max_distance = vehicle_max_travel_distance[v]
    #         max_orders = vehicle_max_order[v]

    #         vehicle_start_time, vehicle_end_time = vehicle_time_windows[v]
    #         current_time = vehicle_start_time
    #         current_route = []
    #         pickup_weight = 0
    #         dropoff_weight = 0
    #         available_weight = vehicle_capacity
    #         total_travel_distance = 0
    #         total_travel_time = timedelta(seconds=0)
    #         order_count = 0

    #         print(f"Assigning customers to Vehicle {v}")

    #         # Start from depot (node 0)
    #         current_node = 0

    #         while unvisited_customers:
    #             # Find the nearest unvisited customer that can be assigned
    #             nearest_customer = None
    #             min_distance = float('inf')

    #             for customer in unvisited_customers:
    #                 if distance_matrix[current_node][customer] < min_distance:
    #                     nearest_customer = customer
    #                     min_distance = distance_matrix[current_node][customer]

    #             if nearest_customer is None:
    #                 break  # No more customers can be assigned

    #             order = nearest_customer
    #             ot = order_types[order - 1]  # Adjust for zero-based indexing
    #             weight = demands[order]

    #             # Calculate the travel time and distance to the nearest customer
    #             travel_time_to_order = timedelta(seconds=time_matrix[current_node][order])
    #             travel_distance_to_order = distance_matrix[current_node][order]

    #             # Calculate the total travel time including serving the customer
    #             total_time_if_assigned = total_travel_time + travel_time_to_order + timedelta(seconds=serve_time[order])
    #             total_distance_if_assigned = total_travel_distance + travel_distance_to_order

    #             # Check if adding this customer exceeds constraints
    #             if (total_distance_if_assigned > vehicle_max_distance or
    #                     total_time_if_assigned > timedelta(seconds=vehicle_max_time) or
    #                     order_count + 1 > max_orders):
    #                 # Skip this customer due to constraints
    #                 skipped_orders.append(order)
    #                 print(f"Skipped Order {order} due to vehicle {v} exceeding constraints.")
    #                 unvisited_customers.remove(order)
    #                 continue

    #             # If the constraints are satisfied, proceed to assign the order
    #             if ot == "pickup":
    #                 # Handle paired pickup orders
    #                 if order in paired_pickup_orders:
    #                     paired_pickup = paired_pickup_orders[order]
    #                     paired_weight = demands[paired_pickup]

    #                     if paired_pickup not in unvisited_customers:
    #                         # Paired pickup already assigned
    #                         skipped_orders.extend([order, paired_pickup])
    #                         print(f"Skipped Pickup Order {order} and Paired Pickup {paired_pickup} as paired pickup is already assigned.")
    #                         unvisited_customers.remove(order)
    #                         continue

    #                     total_pickup_weight = weight + paired_weight
    #                     if pickup_weight + total_pickup_weight <= vehicle_capacity and available_weight >= total_pickup_weight:
    #                         # Assign both pickup and its paired pickup
    #                         current_route.extend([order, paired_pickup])
    #                         pickup_weight += total_pickup_weight
    #                         available_weight -= total_pickup_weight
    #                         unvisited_customers.remove(order)
    #                         unvisited_customers.remove(paired_pickup)
    #                         current_node = paired_pickup  # Move to the paired pickup
    #                         total_travel_time = total_time_if_assigned
    #                         total_travel_distance = total_distance_if_assigned
    #                         order_count += 2
    #                         print(f"Assigned Paired Pickup Orders {order} and {paired_pickup} to current route.")
    #                     else:
    #                         # Cannot assign due to capacity constraints
    #                         skipped_orders.extend([order, paired_pickup])
    #                         print(f"Skipped Paired Pickup Orders {order} and {paired_pickup} due to capacity constraints.")
    #                         unvisited_customers.remove(order)
    #                         continue

    #                 # Handle pickup paired with dropoff
    #                 elif order in pairing_pickup_to_dropoff:
    #                     paired_dropoff = pairing_pickup_to_dropoff[order]
    #                     paired_dropoff_weight = demands[paired_dropoff]

    #                     if paired_dropoff not in unvisited_customers:
    #                         # Paired dropoff already assigned
    #                         skipped_orders.extend([order, paired_dropoff])
    #                         print(f"Skipped Pickup Order {order} and Paired Dropoff {paired_dropoff} as paired dropoff is already assigned.")
    #                         unvisited_customers.remove(order)
    #                         continue

    #                     if pickup_weight + weight + paired_dropoff_weight <= vehicle_capacity and available_weight >= (weight + paired_dropoff_weight):
    #                         # Assign pickup and its paired dropoff
    #                         current_route.extend([order, paired_dropoff])
    #                         unvisited_customers.remove(order)
    #                         available_weight -= weight
    #                         pickup_weight += weight
    #                         unvisited_customers.remove(paired_dropoff)
    #                         current_node = paired_dropoff  # Move to the paired dropoff
    #                         total_travel_time = total_time_if_assigned
    #                         total_travel_distance = total_distance_if_assigned
    #                         order_count += 2
    #                         print(f"Assigned Pickup Order {order} and its Paired Dropoff Order {paired_dropoff} to current route.")
    #                     else:
    #                         # Cannot assign due to capacity constraints
    #                         skipped_orders.extend([order, paired_dropoff])
    #                         print(f"Skipped Pickup Order {order} and Dropoff Order {paired_dropoff} due to capacity constraints.")
    #                         unvisited_customers.remove(order)
    #                         continue

    #                 else:
    #                     # Unpaired pickup
    #                     if pickup_weight + weight <= vehicle_capacity and available_weight >= weight:
    #                         # Assign unpaired pickup
    #                         current_route.append(order)
    #                         pickup_weight += weight
    #                         available_weight -= weight
    #                         unvisited_customers.remove(order)
    #                         current_node = order  # Move to the assigned customer
    #                         total_travel_time = total_time_if_assigned
    #                         total_travel_distance = total_distance_if_assigned
    #                         order_count += 1
    #                         print(f"Assigned Unpaired Pickup Order {order} to current route.")
    #                     else:
    #                         # Cannot assign due to capacity constraints
    #                         skipped_orders.append(order)
    #                         print(f"Skipped Unpaired Pickup Order {order} due to capacity constraints.")
    #                         unvisited_customers.remove(order)
    #                         continue

    #             elif ot == "dropoff":
    #                 # Unpaired dropoff
    #                 if dropoff_weight + weight <= vehicle_capacity and available_weight >= weight:
    #                     # Assign unpaired dropoff
    #                     current_route.append(order)
    #                     dropoff_weight += weight
    #                     unvisited_customers.remove(order)
    #                     current_node = order  # Move to the assigned customer
    #                     total_travel_time = total_time_if_assigned
    #                     total_travel_distance = total_distance_if_assigned
    #                     order_count += 1
    #                     print(f"Assigned Unpaired Dropoff Order {order} to current route.")
    #                 else:
    #                     # Cannot assign due to capacity constraints
    #                     skipped_orders.append(order)
    #                     print(f"Skipped Unpaired Dropoff Order {order} due to capacity constraints.")
    #                     unvisited_customers.remove(order)
    #                     continue

    #         if current_route:
    #             routes[v] = current_route
    #             print(f"Finalized Route for Vehicle {v}: {current_route}")

    #     # Reassign skipped orders to the remaining vehicles in a round-robin manner
    #     unused_vehicles = [v for v in range(num_vehicles) if not routes[v]]
    #     for order in list(skipped_orders):
    #         if not unused_vehicles:
    #             break  # No unused vehicles left
    #         for v in unused_vehicles:
    #             weight = demands[order]
    #             if vehicle_capacities[v] >= weight:
    #                 routes[v].append(order)
    #                 vehicle_capacities[v] -= weight
    #                 skipped_orders.remove(order)
    #                 print(f"Assigned Skipped Order {order} to Vehicle {v}. Remaining Capacity: {vehicle_capacities[v]}")
    #                 break
    #     return routes, skipped_orders






    # def is_valid_route(route, demands, vehicle_capacities, order_volume, vehicle_index, vehicle_volume,
    #                 time_matrix, vehicle_max_travel_time, serve_time, vehicle_max_weight, order_types,
    #                 pairing_pickup_to_dropoff, paired_pickup_orders, paired_dropoff_orders,
    #                 vehicle_time_windows, order_time_windows, current_travel_time,
    #                 max_travel_time, current_travel_distance, max_travel_distance,
    #                 current_order_count, max_order_count):
    #     # Initialize variables
    #     vehicle_start_time, vehicle_end_time = vehicle_time_windows[vehicle_index]
    #     current_time = vehicle_start_time
    #     available_weight = vehicle_capacities[vehicle_index]
    #     current_volume = 0
    #     pickup_weight = 0
    #     dropoff_weight = 0
    #     assigned_dropoffs = set()
    #     assigned_pickups = set()
    #     total_travel_time_increment = timedelta(seconds=0)
    #     total_travel_distance_increment = 0
    #     order_count_increment = 0

    #     for i, order in enumerate(route):
    #         if order in assigned_dropoffs or order in assigned_pickups:
    #             continue

    #         ot = order_types[order - 1]  # Adjust for zero-based indexing
    #         weight = demands[order]

    #         if ot == "pickup":
    #             if order in paired_pickup_orders:
    #                 paired_pickup = paired_pickup_orders[order]
    #                 paired_weight = demands[paired_pickup]

    #                 if paired_pickup not in assigned_pickups:
    #                     total_pickup_weight = weight + paired_weight
    #                     if pickup_weight + total_pickup_weight <= vehicle_capacities[vehicle_index] and available_weight >= total_pickup_weight:
    #                         assigned_pickups.update([order, paired_pickup])
    #                         pickup_weight += total_pickup_weight
    #                         available_weight -= total_pickup_weight
    #                     else:
    #                         return False, None, None  # Capacity exceeded with paired pickup
    #             elif order in pairing_pickup_to_dropoff:
    #                 paired_dropoff = pairing_pickup_to_dropoff[order]
    #                 paired_dropoff_weight = demands[paired_dropoff]

    #                 if paired_dropoff not in assigned_dropoffs:
    #                     if pickup_weight + weight + paired_dropoff_weight <= vehicle_capacities[vehicle_index] and available_weight >= (weight + paired_dropoff_weight):
    #                         assigned_pickups.add(order)
    #                         assigned_dropoffs.add(paired_dropoff)
    #                         pickup_weight += weight
    #                         available_weight -= (weight + paired_dropoff_weight)
    #                     else:
    #                         return False, None, None  # Capacity exceeded with pickup and its paired dropoff
    #             else:
    #                 if pickup_weight + weight <= vehicle_capacities[vehicle_index] and available_weight >= weight:
    #                     assigned_pickups.add(order)
    #                     pickup_weight += weight
    #                     available_weight -= weight
    #                 else:
    #                     return False, None, None  # Capacity exceeded with unpaired pickup

    #         elif ot == "dropoff":
    #             paired_dropoff = None
    #             for key, value in paired_dropoff_orders.items():
    #                 if value == order:
    #                     paired_dropoff = key
    #                     break
    #                 elif key == order:
    #                     paired_dropoff = value
    #                     break

    #             if paired_dropoff:
    #                 paired_dropoff_weight = demands[paired_dropoff]

    #                 if paired_dropoff not in assigned_dropoffs:
    #                     if dropoff_weight + weight + paired_dropoff_weight <= vehicle_capacities[vehicle_index] and available_weight >= (weight + paired_dropoff_weight):
    #                         assigned_dropoffs.update([order, paired_dropoff])
    #                         dropoff_weight += (weight + paired_dropoff_weight)
    #                     else:
    #                         return False, None, None  # Capacity exceeded with paired dropoff
    #             else:
    #                 if dropoff_weight + weight <= vehicle_capacities[vehicle_index] and available_weight >= weight:
    #                     assigned_dropoffs.add(order)
    #                     dropoff_weight += weight
    #                 else:
    #                     return False, None, None  # Capacity exceeded with unpaired dropoff

    #         # Update order count
    #         order_count_increment += 1
    #         if order_count_increment > max_order_count:
    #             return False, None, None  # Exceeds max order count

    #         # Update travel time and distance increment
    #         if i == 0:
    #             # From depot to first customer
    #             travel_time = timedelta(seconds=time_matrix[0][order])
    #             travel_distance = distance_matrix[0][order]
    #         else:
    #             prev_order = route[i - 1]
    #             travel_time = timedelta(seconds=time_matrix[prev_order][order])
    #             travel_distance = distance_matrix[prev_order][order]

    #         current_time += travel_time
    #         total_travel_time_increment += travel_time
    #         total_travel_distance_increment += travel_distance

    #         # Check time window constraints
    #         order_start_time, order_end_time = order_time_windows[order - 1]
    #         if current_time > order_end_time:
    #             return False, None, None  # Cannot arrive before time window closes
    #         if current_time < order_start_time:
    #             current_time = order_start_time  # Wait until time window opens

    #         # Add service time
    #         service_time = timedelta(seconds=serve_time[order])
    #         current_time += service_time
    #         total_travel_time_increment += service_time

    #         # Check vehicle's operating time window
    #         if current_time > vehicle_end_time:
    #             return False, None, None  # Exceeds vehicle's operating time window

    #     # After processing all orders, check final constraints on travel time and distance
    #     if (current_travel_time + total_travel_time_increment > timedelta(seconds=max_travel_time) or
    #             current_travel_distance + total_travel_distance_increment > max_travel_distance):
    #         return False, None, None

    #     return True, total_travel_time_increment, total_travel_distance_increment


    # def calculate_insertion_cost(distance_matrix, current_route, customer, position):
    #     if position == 0:
    #         prev_node = 0  # Depot
    #     else:
    #         prev_node = current_route[position - 1]
    #     if position == len(current_route):
    #         next_node = 0  # Depot or end of route
    #     else:
    #         next_node = current_route[position]

    #     additional_cost = (
    #         distance_matrix[prev_node][customer] +
    #         distance_matrix[customer][next_node] -
    #         distance_matrix[prev_node][next_node]
    #     )
    #     return additional_cost
   



    
    

    csv_data = [["Vehicle", "Route", "Route Total Orders", "Dropoff Orders", "Pickup Orders", "Route Total Weight (kg)", "Route Total Volume (cm³)", "Route Cost", "Vehicle Fixed Cost", "Route Distance Cost", "Route Time Cost", "Route Distance (km)", "Vehicle Contract Distance (km)", "Detour Distance (km)", "Vehicle Cost per Unit Distance", "Vehicle Variable Cost per Unit Distance", "Route Time (s)", "Vehicle Contract Time (s)", "Overtime (s)", "Vehicle Cost per Unit Time", "Vehicle Variable Cost per Unit Time", "Vehicle Max Travel Distance (km)", "Vehicle Max Available Time (s)", "Vehicle Max Weight (kg)", "Vehicle Max Volume (cm³)", "Vehicle Distance Utilization (%)", "Vehicle Time Utilization (%)", "Vehicle Weight Utilization (%)", "Vehicle Volume Utilization (%)"]]

    total_cost = 0
    total_distance = 0
    total_time = 0
    

    for i, route in enumerate(final_routes):
        if len(route) > 0:
            route_cost, route_distance, route_time = calculate_route_cost(route, i, distance_matrix, time_matrix)
            

            
            route_total_orders = sum(1 for j in route if j != 0)  # Exclude depot

            pickup_orders = sum(1 for j in route if order_types[j - 1] == 'pickup') if len(route) > 0 else 0
            dropoff_orders = sum(1 for j in route if order_types[j - 1] == 'dropoff') if len(route) > 0 else 0
            
            route_total_weight = sum(demands[j] for j in route if j != 0)
            route_total_volume = sum(order_volume[j] for j in route if j != 0)
            
            vehicle_distance_utilization = (route_distance / vehicle_max_travel_distance[i]) * 100
            vehicle_time_utilization = (route_time / vehicle_max_travel_time[i]) * 100
            vehicle_weight_utilization = (route_total_weight / vehicle_capacities[i]) * 100
            vehicle_volume_utilization = (route_total_volume / vehicle_volume[i]) * 100
            
            detour_distance = max(0, route_distance - contract_distance[i])
            overtime = max(0, route_time - contract_time[i])
            
            csv_data.append([
                vehicle_contract_code[i],
                " -> ".join(map(str, route)),
                route_total_orders,
                dropoff_orders,
                pickup_orders,
                route_total_weight,
                route_total_volume,
                route_cost,
                fixed_cost[i],
                cost_per_km[i]*route_distance,
                (route_time * cost_per_minute[i]),
                route_distance,
                contract_distance[i],
                detour_distance,
                cost_per_km[i],
                cost_per_km[i],
                route_time,
                contract_time[i],
                overtime,
                cost_per_minute[i],
                variable_cost_per_minute[i],
                vehicle_max_travel_distance[i],
                vehicle_max_travel_time[i],
                vehicle_max_weight[i],
                vehicle_volume[i],
                vehicle_distance_utilization,
                vehicle_time_utilization,
                vehicle_weight_utilization,
                vehicle_volume_utilization
            ])

            total_cost += route_cost
            total_distance += route_distance
            total_time += route_time


    # Save the CSV results in a specific folder
    output_dir = os.path.join(result_folder, os.path.basename(file_path).replace('.json', ''))
    os.makedirs(output_dir, exist_ok=True)

    csv_file_path = os.path.join(output_dir, 'results.csv')
    with open(csv_file_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(csv_data)

    end_time = time.time()


    print(f"Processed file: {file_path}")
    print(f"Total cost: {total_cost}")
    print(f"Total distance: {total_distance}")
    print(f"Total time: {total_time}")
    print(f"Time taken: {end_time - start_time} seconds")
    print(f"Results saved to: {csv_file_path}")
 
    end_time = time.time()
    duration = end_time - start_time

    # Prepare result output
    result = {
        "routes": final_routes,
        "duration": duration
    }

    result_file = os.path.join(result_folder, "result.json")
    with open(result_file, 'w') as f:
        json.dump(result, f, indent=4)

    print(f"Results saved to {result_file}")

if __name__ == "__main__":
    input_file = 'data1.json'  # Replace with your input file path
    result_folder = '/home/delhivery/Desktop/vrp_project/data'  # Replace with your result folder path

    if not os.path.exists(result_folder):
        os.makedirs(result_folder)

    process_vrp_file(input_file, result_folder)
    

Assigned Unpaired Dropoff Order 2 to current route.
Assigned Unpaired Dropoff Order 1 to current route.
Assigned Unpaired Dropoff Order 3 to current route.
Skipped Unpaired Dropoff Order 4 due to capacity constraints.
Assigned Unpaired Dropoff Order 5 to current route.
Assigned Unpaired Dropoff Order 6 to current route.
Skipped Unpaired Dropoff Order 7 due to capacity constraints.
Skipped Unpaired Dropoff Order 8 due to capacity constraints.
Finalized Route for Vehicle 0: [2, 1, 3, 5, 6]
All customers have been assigned. No unvisited customers left for vehicle 1.
Assigned Unpaired Dropoff Order 4 to current route.
Skipped Unpaired Dropoff Order 7 due to capacity constraints.
Skipped Unpaired Dropoff Order 8 due to capacity constraints.
Finalized Route for Vehicle 1: [4]
Assigned Unpaired Dropoff Order 4 to current route.
Skipped Unpaired Dropoff Order 7 due to capacity constraints.
Skipped Unpaired Dropoff Order 8 due to capacity constraints.
Finalized Route for Vehicle 2: [4]
([[2, 1,

TypeError: list indices must be integers or slices, not list

In [2]:
from datetime import datetime